In this notebook we will analyze the plate reader data for validation of the split pi-protein system, and generate the plots used in making Supplementary Figure 4.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize
import murraylab_tools.biotek as btek
import itertools

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from cycler import cycler
import numpy as np


sns.set_context("talk", font_scale=1.5, rc={"lines.linewidth": 1.5})
sns.set_style("ticks")
sns.set_style({"xtick.direction": "in","ytick.direction": "in"})

mpl.rc('axes', prop_cycle=(cycler('color', ['r', 'k', 'b','g','y','m','c']) ))

mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42

tw = 1.5
sns.set_style({"xtick.major.size": 3, "ytick.major.size": 3,
               "xtick.minor.size": 2, "ytick.minor.size": 2,
               'axes.labelsize': 16, 'axes.titlesize': 16,
               'xtick.major.width': tw, 'xtick.minor.width': tw,
               'ytick.major.width': tw, 'ytick.minor.width': tw})

mpl.rc('xtick', labelsize=14) 
mpl.rc('ytick', labelsize=14)
mpl.rc('axes', linewidth=1.5)
mpl.rc('legend', fontsize=14)
mpl.rc('legend', frameon=False)
mpl.rc('figure', figsize=(8.5,15))
%matplotlib qt

In [2]:
# generate csvs with tidy data frames
btek.tidy_biotek_data('./20210825_split_pir_validation_plate1.csv',convert_to_uM=False,volume=300,supplementary_filename='20210825_metadata.csv',)
btek.tidy_biotek_data('./20210825_split_pir_validation_plate2.csv',convert_to_uM=False,volume=300,supplementary_filename='20210825_metadata.csv')

/Users/rorywilliams/opt/anaconda3/lib/python3.7/site-packages/murraylab_tools/biotek/biotek.py:473: UserWarning: No supplementary data for well H1; throwing out data for that well.
  % well_name)
/Users/rorywilliams/opt/anaconda3/lib/python3.7/site-packages/murraylab_tools/biotek/biotek.py:473: UserWarning: No supplementary data for well H2; throwing out data for that well.
  % well_name)
/Users/rorywilliams/opt/anaconda3/lib/python3.7/site-packages/murraylab_tools/biotek/biotek.py:473: UserWarning: No supplementary data for well H3; throwing out data for that well.
  % well_name)
/Users/rorywilliams/opt/anaconda3/lib/python3.7/site-packages/murraylab_tools/biotek/biotek.py:473: UserWarning: No supplementary data for well H4; throwing out data for that well.
  % well_name)
/Users/rorywilliams/opt/anaconda3/lib/python3.7/site-packages/murraylab_tools/biotek/biotek.py:473: UserWarning: No supplementary data for well H5; throwing out data for that well.
  % well_name)
/Users/rorywilliams/

/Users/rorywilliams/opt/anaconda3/lib/python3.7/site-packages/murraylab_tools/biotek/biotek.py:473: UserWarning: No supplementary data for well H1; throwing out data for that well.
  % well_name)
/Users/rorywilliams/opt/anaconda3/lib/python3.7/site-packages/murraylab_tools/biotek/biotek.py:473: UserWarning: No supplementary data for well H2; throwing out data for that well.
  % well_name)
/Users/rorywilliams/opt/anaconda3/lib/python3.7/site-packages/murraylab_tools/biotek/biotek.py:473: UserWarning: No supplementary data for well H3; throwing out data for that well.
  % well_name)
/Users/rorywilliams/opt/anaconda3/lib/python3.7/site-packages/murraylab_tools/biotek/biotek.py:473: UserWarning: No supplementary data for well H4; throwing out data for that well.
  % well_name)
/Users/rorywilliams/opt/anaconda3/lib/python3.7/site-packages/murraylab_tools/biotek/biotek.py:473: UserWarning: No supplementary data for well H5; throwing out data for that well.
  % well_name)
/Users/rorywilliams/

In [3]:
# load data and generate data frames with fluorescence normalized by OD, can read all tidy dfs in cell below
p1 = pd.read_csv('20210825_split_pir_validation_plate1_tidy.csv')
p1_norm = btek.normalize(p1)
p2 = pd.read_csv('20210825_split_pir_validation_plate2_tidy.csv')
p2_norm = btek.normalize(p2)
p1_norm.to_csv('20210825_split_pir_validation_plate1_tidy_norm.csv')
p2_norm.to_csv('20210825_split_pir_validation_plate2_tidy_norm.csv')

In [4]:
p1 = pd.read_csv('20210825_split_pir_validation_plate1_tidy.csv')
p1_norm = pd.read_csv('20210825_split_pir_validation_plate1_tidy_norm.csv')
p2 = pd.read_csv('20210825_split_pir_validation_plate2_tidy.csv')
p2_norm = pd.read_csv('20210825_split_pir_validation_plate2_tidy_norm.csv')

In [5]:
# get endpoint data
p1_OD = p1.loc[p1.Channel=='OD600',:]
p1_RFP = p1.loc[(p1.Channel=='mScarlet')&\
                (p1.Gain==100),:]
p1_GFP = p1.loc[(p1.Channel=='sfGFP')&\
                (p1.Gain==61),:]
p1_norm_RFP = p1_norm.loc[(p1_norm.Channel=='mScarlet')&\
                (p1_norm.Gain==100),:]
p1_norm_GFP = p1_norm.loc[(p1_norm.Channel=='sfGFP')&\
                (p1_norm.Gain==61),:]
p2_OD = p2.loc[p2.Channel=='OD600',:]
p2_RFP = p2.loc[(p2.Channel=='mScarlet')&\
                (p2.Gain==100),:]
p2_GFP = p2.loc[(p2.Channel=='sfGFP')&\
                (p2.Gain==61),:]
p2_norm_RFP = p2_norm.loc[(p2_norm.Channel=='mScarlet')&\
                (p2_norm.Gain==100),:]
p2_norm_GFP = p2_norm.loc[(p2_norm.Channel=='sfGFP')&\
                (p2_norm.Gain==61),:]
p2_OD_endpoint = p2_OD.loc[p2_OD['Time (hr)']==p2_OD['Time (hr)'].max(),:]
p2_RFP_endpoint = p2_RFP.loc[p2_RFP['Time (hr)']==p2_RFP['Time (hr)'].max(),:]
p2_GFP_endpoint = p2_GFP.loc[p2_GFP['Time (hr)']==p2_GFP['Time (hr)'].max(),:]
p2_norm_RFP_endpoint = p2_norm_RFP.loc[p2_norm_RFP['Time (hr)']==p2_norm_RFP['Time (hr)'].max(),:]
p2_norm_GFP_endpoint = p2_norm_GFP.loc[p2_norm_GFP['Time (hr)']==p2_norm_GFP['Time (hr)'].max(),:]

In [6]:
p2_norm_RFP_endpoint.head()

,Unnamed: 0,index,Channel,Gain,Time (sec),Time (hr),Well,Measurement,Units,Excitation,Emission,strain,kan,chlor,carb,las,sal,rep,ChanStr
30576,30576,24444,mScarlet,100,43696,12.137778,A1,32257.901391,AFU/OD,565,595,marionette + p15a split pir 1,+,+,-,0.03,0,1,mScarlet100565595
30577,30577,24445,mScarlet,100,43696,12.137778,A2,35814.121037,AFU/OD,565,595,marionette + p15a split pir 1,+,+,-,0.03,0,2,mScarlet100565595
30578,30578,24446,mScarlet,100,43696,12.137778,A3,33612.323492,AFU/OD,565,595,marionette + p15a split pir 1,+,+,-,0.03,0,3,mScarlet100565595
30579,30579,24447,mScarlet,100,43696,12.137778,A4,32585.025381,AFU/OD,565,595,marionette + p15a split pir 1,+,+,-,0.10,0,1,mScarlet100565595
30580,30580,24448,mScarlet,100,43696,12.137778,A5,41397.984887,AFU/OD,565,595,marionette + p15a split pir 1,+,+,-,0.10,0,2,mScarlet100565595


In [7]:
RFP_plot = p2_norm_RFP_endpoint.copy()
GFP_plot = p2_norm_GFP_endpoint.copy()
strains_plot = ['pir excision', 'pirR excision', 'pirL excision','diff + p15a split pir 1']
for strain in RFP_plot.strain.unique():
    if strain not in strains_plot:
        RFP_plot.drop(index=RFP_plot.loc[RFP_plot.strain==strain].index.values,inplace=True)
        GFP_plot.drop(index=GFP_plot.loc[GFP_plot.strain==strain].index.values,inplace=True)

Below we generate the plot shown in Supplementary Figure 4, note the order has been changed in Affinity Designer in the published figure.

In [8]:
sns.set_palette(['black','#d73027','#fc8d59','#91bfdb','#4575b4'])
fig, ax = plt.subplots(2,1,sharey='row',sharex='col',figsize=(15,7))  

sns.swarmplot(ax=ax[0],x="strain", y="Measurement", hue="sal", data=RFP_plot.loc[RFP_plot.las==0.03,:],dodge=True,size=6)
ax[0].get_legend().set_visible(False)
ax[0].set_xlabel("")
sns.swarmplot(ax=ax[1],x="strain", y="Measurement", hue="sal", data=GFP_plot.loc[GFP_plot.las==0.03,:],dodge=True,size=6)
#     sns.boxplot(ax=ax[1,i],x="las", y="Measurement", hue="sal", data=data,dodge=True,saturation=0.75,linewidth=1.25,width=0.2)
#     sns.barplot(ax=ax[1,i],x="las", y="Measurement", hue="sal", data=data,capsize=0.1,errwidth=1.25,alpha=0.7,ci='sd')
ax[1].get_legend().set_visible(False)
ax[1].set_xlabel("")
#     ax[i,j].set_ylim(0,60000)

ax[0].set_yscale('log')
ax[0].set_ylim(1e3,1e5)
ax[1].set_yscale('log')
ax[1].set_ylim(1e2,1e4)
ax[0].set_ylabel('mScarlet/OD600 (a.u)',fontsize=14)
ax[1].set_ylabel('sfGFP/OD600 (a.u)',fontsize=14)

plt.savefig('./splitpirval_plot_v2.pdf')